# Prédiction de la consommation en carburant de voitures récentes.


### Rouler cette cellule pour installer les packages necessaires

In [89]:
using Pkg

Pkg.add([
    "CSV",                # Pour charger les fichiers CSV
    "DataFrames",         # Pour utiliser la structure DataFrame pour stocker les données
    "Distributions",      # Pour utiliser les lois de probabilités classiques
    "Gadfly",             # Pour tracer des graphiques
    "Statistics",         # Pour les fonctions statistiques de base
    "MLJ",                # Pour le machine learning
    "CategoricalArrays",  # Pour manipuler les données catégorielles
    "StatsBase"           # Pour les fonctions statistiques avancées
])

   Resolving package versions...
    Updating `C:\Users\Dimi\.julia\environments\v1.10\Project.toml`
  [324d7699] + CategoricalArrays v0.10.8
⌅ [2913bbd2] + StatsBase v0.33.21
  [10745b16] + Statistics v1.10.0
  No Changes to `C:\Users\Dimi\.julia\environments\v1.10\Manifest.toml`


### Importation de librairies

In [90]:
using CSV              # Pour charger les fichiers CSV
using DataFrames       # Pour utiliser la structure DataFrame pour stocker les données
using Distributions    # Pour utiliser les lois de probabilités classiques
using Gadfly           # Pour tracer des graphiques
using Statistics       # Pour les functions statistiques de base
using MLJ
using CategoricalArrays
using StatsBase

### Chargement des données train et test

In [92]:
train_data = CSV.read("train.csv", DataFrame);
test_data = CSV.read("test.csv", DataFrame);

In [93]:
first(train_data, 10)

Row,annee,type,nombre_cylindres,cylindree,transmission,boite,consommation
,Int64,String31,Int64,String3,String15,String15,String31
1,2023,voiture_moyenne,8,"4,4",integrale,automatique,"13,8358823529412"
2,2020,VUS_petit,4,2,integrale,automatique,"9,80041666666667"
3,2021,voiture_compacte,6,"3,3",propulsion,automatique,"11,7605"
4,2023,voiture_deux_places,8,5,integrale,automatique,"13,0672222222222"
5,2022,voiture_moyenne,8,"4,4",integrale,automatique,"13,8358823529412"
6,2022,voiture_moyenne,8,"4,4",integrale,automatique,"13,8358823529412"
7,2022,voiture_minicompacte,3,"1,5",traction,automatique,"7,3503125"
8,2024,voiture_minicompacte,3,"1,5",traction,manuelle,"7,58741935483871"
9,2020,VUS_standard,6,"3,8",integrale,automatique,"11,2004761904762"


In [94]:
first(test_data, 10)

Row,annee,type,nombre_cylindres,cylindree,transmission,boite
,Int64,String31,Int64,String3,String15,String15
1,2014,voiture_moyenne,4,"2,5",traction,manuelle
2,2014,voiture_moyenne,4,"2,5",traction,automatique
3,2014,VUS_petit,4,"2,5",traction,automatique
4,2014,VUS_petit,4,2,4x4,automatique
5,2014,voiture_sous_compacte,8,"5,8",propulsion,manuelle
6,2014,voiture_sous_compacte,8,5,propulsion,automatique
7,2014,voiture_sous_compacte,8,5,propulsion,manuelle
8,2014,VUS_petit,4,"2,4",4x4,automatique
9,2014,VUS_petit,6,"3,5",integrale,automatique


In [95]:
# Transformer les colonnes supposer etre numeric de String en Float pour les données train et test
train_data.cylindree = parse.(Float64, replace.(train_data.cylindree, "," => "."));
train_data.consommation = parse.(Float64, replace.(train_data.consommation, "," => "."));
test_data.cylindree = parse.(Float64, replace.(test_data.cylindree, "," => "."));

In [96]:
# categorical est utilisé pour encoder les variables catégorielles afin de les utiliser dans les modèles
train_data.type = categorical(train_data.type);
train_data.transmission = categorical(train_data.transmission);
train_data.boite = categorical(train_data.boite);

test_data.type = categorical(test_data.type);
test_data.transmission = categorical(test_data.transmission);
test_data.boite = categorical(test_data.boite);

In [97]:
first(train_data, 10)

Row,annee,type,nombre_cylindres,cylindree,transmission,boite,consommation
,Int64,Cat…,Int64,Float64,Cat…,Cat…,Float64
1,2023,voiture_moyenne,8,4.4,integrale,automatique,13.8359
2,2020,VUS_petit,4,2.0,integrale,automatique,9.80042
3,2021,voiture_compacte,6,3.3,propulsion,automatique,11.7605
4,2023,voiture_deux_places,8,5.0,integrale,automatique,13.0672
5,2022,voiture_moyenne,8,4.4,integrale,automatique,13.8359
6,2022,voiture_moyenne,8,4.4,integrale,automatique,13.8359
7,2022,voiture_minicompacte,3,1.5,traction,automatique,7.35031
8,2024,voiture_minicompacte,3,1.5,traction,manuelle,7.58742
9,2020,VUS_standard,6,3.8,integrale,automatique,11.2005


In [98]:
first(test_data, 10)

Row,annee,type,nombre_cylindres,cylindree,transmission,boite
,Int64,Cat…,Int64,Float64,Cat…,Cat…
1,2014,voiture_moyenne,4,2.5,traction,manuelle
2,2014,voiture_moyenne,4,2.5,traction,automatique
3,2014,VUS_petit,4,2.5,traction,automatique
4,2014,VUS_petit,4,2.0,4x4,automatique
5,2014,voiture_sous_compacte,8,5.8,propulsion,manuelle
6,2014,voiture_sous_compacte,8,5.0,propulsion,automatique
7,2014,voiture_sous_compacte,8,5.0,propulsion,manuelle
8,2014,VUS_petit,4,2.4,4x4,automatique
9,2014,VUS_petit,6,3.5,integrale,automatique
